In [ ]:
"""
================================================================================
PROYEK      : Estimasi Debit Sungai Metode CMHC Berbasis Citra Sentinel-2
MODUL       : Langkah B - Pre-processing, Outlier Removal, & Klasifikasi Jenks
DESKRIPSI   : Script ini melakukan:
              1. Pembersihan data debit in-situ (deteksi outlier IQR/Z-score).
              2. Harmonisasi data debit dengan tanggal citra satelit yang tersedia.
              3. Klasifikasi debit ke dalam kelas (Rendah/Menengah/Tinggi)
                 menggunakan metode statistik Jenks Natural Breaks.
              4. Visualisasi distribusi data dan ekspor dataset bersih.
================================================================================

DEPENDENSI:
  - pandas
  - numpy
  - matplotlib
  - seaborn
  - jenkspy (pip install jenkspy)
"""

# ==============================================================================
# 1. IMPOR LIBRARY
# ==============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import calendar
import os
import json
import warnings
import traceback
from pathlib import Path
from google.colab import drive

# Cek & Instalasi Library Khusus
try:
    import jenkspy
    print("✅ Library 'jenkspy' siap digunakan.")
except ImportError:
    print("⚠️ Library 'jenkspy' tidak ditemukan. Menginstal...")
    import subprocess
    subprocess.check_call(["pip", "install", "jenkspy"])
    import jenkspy
    print("✅ Instalasi 'jenkspy' selesai.")

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# ==============================================================================
# 2. KONFIGURASI GLOBAL
# ==============================================================================

# Folder Kerja (Sesuaikan jika dijalankan di lokal/bukan Colab)
GEE_EXPORT_FOLDER = '/content/drive/MyDrive/GEE_Exports_Tesis_Master/'

# Daftar Stasiun Kajian
STATION_LIST = ['Kalibawang', 'Bendungan', 'Kedungmiri']

# Mapping Nama untuk Visualisasi
STATION_FULL_NAMES = {
    'Kalibawang': 'Sungai Progo (Stasiun Kalibawang)',
    'Kedungmiri': 'Sungai Oyo (Stasiun Kedungmiri)',
    'Bendungan' : 'Sungai Serang (Stasiun Bendungan)'
}

# Mapping ZOM (Zona Musim)
ROI_TO_ZOM = {
    'Bendungan' : 'ZOM_06',
    'Kalibawang': 'ZOM_01',
    'Kedungmiri': 'ZOM_07'
}

# Periode Pembagian Data (Split Temporal)
START_LATIH_STR = '2019-01-01'
END_LATIH_STR   = '2021-12-31'
START_UJI_STR   = '2022-01-01'
END_UJI_STR     = '2024-12-31'

# Parameter Statistik Outlier
IQR_MULT = 1.5  # Batas Interquartile Range
Z_TH     = 3.0  # Batas Z-Score

# Database Musim Presisi (Disinkronkan dengan Grafik Curah Hujan & ZOM)
RAINY_SEASON_DB = [
    # KALIBAWANG (ZOM_01)
    { 'zom':'ZOM_01', 'start':'2019-11-01', 'end':'2020-05-20' },
    { 'zom':'ZOM_01', 'start':'2020-10-11', 'end':'2021-06-10' },
    { 'zom':'ZOM_01', 'start':'2021-10-11', 'end':'2022-06-10' },
    { 'zom':'ZOM_01', 'start':'2022-10-01', 'end':'2023-05-20' },
    { 'zom':'ZOM_01', 'start':'2023-11-11', 'end':'2024-05-20' },
    { 'zom':'ZOM_01', 'start':'2024-10-11', 'end':'2024-12-31' },
    # BENDUNGAN (ZOM_06)
    { 'zom':'ZOM_06', 'start':'2019-10-21', 'end':'2020-05-01' },
    { 'zom':'ZOM_06', 'start':'2020-10-11', 'end':'2021-04-30' },
    { 'zom':'ZOM_06', 'start':'2021-10-11', 'end':'2022-05-20' },
    { 'zom':'ZOM_06', 'start':'2022-10-01', 'end':'2023-06-30' },
    { 'zom':'ZOM_06', 'start':'2023-11-11', 'end':'2024-05-01' },
    { 'zom':'ZOM_06', 'start':'2024-10-21', 'end':'2024-12-31' },
    # KEDUNGMIRI (ZOM_07)
    { 'zom':'ZOM_07', 'start':'2019-11-01', 'end':'2020-04-30' },
    { 'zom':'ZOM_07', 'start':'2020-10-11', 'end':'2021-04-20' },
    { 'zom':'ZOM_07', 'start':'2021-10-11', 'end':'2022-05-10' },
    { 'zom':'ZOM_07', 'start':'2022-10-01', 'end':'2023-06-30' },
    { 'zom':'ZOM_07', 'start':'2023-11-01', 'end':'2024-05-10' },
    { 'zom':'ZOM_07', 'start':'2024-11-01', 'end':'2024-12-31' }
]

# Variabel Global untuk Ringkasan Akhir
RINGKASAN_ROWS = []
TAG_LATIH = f"{START_LATIH_STR[:4]}_{END_LATIH_STR[:4]}"
TAG_UJI   = f"{START_UJI_STR[:4]}_{END_UJI_STR[:4]}"
VISUAL_COLORS = {'Hujan': '#3498db', 'Kemarau': '#9b59b6'}
MONTH_ORDER = list(calendar.month_abbr)[1:]

# ==============================================================================
# 3. DEFINISI FUNGSI
# ==============================================================================

def mount_google_drive():
    """Menghubungkan ke Google Drive (Khusus Colab)."""
    try:
        drive.mount('/content/drive', force_remount=True)
    except Exception:
        pass # Abaikan jika bukan di environment Colab atau sudah mount

def standardize_date_column(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
    """Mengubah format kolom tanggal menjadi datetime standar."""
    if col_name not in df.columns:
        raise ValueError(f"Kolom tanggal '{col_name}' tidak ditemukan.")
    df['Tanggal'] = pd.to_datetime(df[col_name], errors='coerce')
    if col_name != 'Tanggal': df = df.drop(columns=[col_name])
    return df.dropna(subset=['Tanggal'])

def standardize_debit_column(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
    """Membersihkan format angka pada kolom debit (koma ke titik)."""
    if col_name not in df.columns:
        raise ValueError(f"Kolom debit '{col_name}' tidak ditemukan.")
    if df[col_name].dtype == object:
        df[col_name] = df[col_name].astype(str).str.replace(",", ".", regex=False)
    df['Debit_Mentah'] = pd.to_numeric(df[col_name], errors='coerce')
    if col_name != 'Debit_Mentah': df = df.drop(columns=[col_name])
    return df.dropna(subset=['Debit_Mentah'])

def get_season_label(date_obj, intervals_list):
    """Menentukan label musim berdasarkan tanggal dan interval ZOM."""
    if not pd.notna(date_obj): return 'Lainnya'
    for start, end in intervals_list:
        if start <= date_obj <= end: return 'Hujan'
    return 'Kemarau'

def flag_outliers(df, value_col, iqr_mult, z_th):
    """Menandai data outlier menggunakan metode IQR dan Z-Score."""
    s = df[value_col]
    q1, q3 = s.quantile([0.25, 0.75])
    iqr = q3 - q1
    lower = q1 - (iqr_mult * iqr)
    upper = q3 + (iqr_mult * iqr)
    mean = s.mean()
    std = s.std()

    df_out = df.copy()
    df_out['zscore'] = (df_out[value_col] - mean) / std if std != 0 else 0
    df_out['is_outlier'] = (df_out[value_col] < lower) | \
                           (df_out[value_col] > upper) | \
                           (df_out['zscore'].abs() > z_th)
    return df_out

def calculate_jenks_breaks(df_data, data_col, label):
    """Menghitung ambang batas (threshold) kelas menggunakan Jenks Natural Breaks."""
    data_1d = df_data[data_col].dropna().values

    if len(data_1d) < 5:
        print(f"      ❌ Data terlalu sedikit (n={len(data_1d)}). Gagal hitung Jenks.")
        return None

    try:
        breaks = jenkspy.jenks_breaks(data_1d, n_classes=3)
        q1, q2 = breaks[1], breaks[2]

        # Hitung distribusi
        rendah = np.sum(data_1d < q1)
        menengah = np.sum((data_1d >= q1) & (data_1d < q2))
        tinggi = np.sum(data_1d >= q2)

        print(f"      ✅ Jenks {label}: q1={q1:.3f}, q2={q2:.3f} | Dist: R={rendah}, M={menengah}, T={tinggi}")
        return {'Jenks': {'q1': q1, 'q2': q2}}
    except Exception as e:
        print(f"      ❌ Error Jenks: {e}")
        return None

def assign_debit_class(value, q1, q2):
    """Mengklasifikasikan nilai debit berdasarkan threshold."""
    if value < q1: return 'Rendah'
    elif value < q2: return 'Menengah'
    else: return 'Tinggi'

def visualize_distribution_with_outliers(df, station_name, output_folder):
    """Membuat dan menyimpan grafik batang distribusi citra per musim."""
    print(f"   📊 Membuat Grafik Distribusi...")

    df_plot = df.copy()
    df_plot['Year'] = df_plot['Tanggal'].dt.year
    df_plot['Month'] = df_plot['Tanggal'].dt.month
    df_plot['Month_Name'] = df_plot['Tanggal'].dt.strftime('%b')

    # Filter hanya data valid (bukan outlier) untuk plot
    df_valid = df_plot[~df_plot['is_outlier']]
    summary = df_valid.groupby(['Year', 'Month', 'Month_Name', 'Musim']).size().reset_index(name='Count')

    # Setup Plot
    years = sorted(df_plot['Year'].unique())
    n_years = len(years)
    rows = (n_years + 1) // 2

    fig, axes = plt.subplots(rows, 2, figsize=(16, 4 * rows), sharey=False)

    full_name = STATION_FULL_NAMES.get(station_name, f"Stasiun {station_name}")
    title = f'Distribusi Jumlah Citra Valid {min(years)}–{max(years)}\n{full_name}'
    fig.suptitle(title, fontsize=16, fontweight='bold', y=0.98)

    axes = axes.flatten()
    global_max = summary['Count'].max() if not summary.empty else 0

    for i, year in enumerate(years):
        ax = axes[i]
        data_year = summary[summary['Year'] == year]

        # Template bulan lengkap
        months_df = pd.DataFrame({'Month': range(1, 13), 'Month_Name': MONTH_ORDER})
        data_merged = pd.merge(months_df, data_year, on=['Month', 'Month_Name'], how='left')
        data_merged['Count'] = data_merged['Count'].fillna(0)

        sns.barplot(data=data_merged, x='Month_Name', y='Count', hue='Musim',
                    palette=VISUAL_COLORS, ax=ax, dodge=False, edgecolor='black', linewidth=1)

        ax.set_title(f'Tahun {year}', fontweight='bold', loc='left', fontsize=12)
        ax.set_xlabel('')
        ax.set_ylabel('Jumlah Citra' if i % 2 == 0 else '')
        ax.set_ylim(0, global_max + 2)
        ax.grid(axis='y', linestyle='--', alpha=0.5)
        ax.legend_.remove()

        # Anotasi Angka
        for p in ax.patches:
            height = p.get_height()
            if height > 0:
                ax.annotate(f'{int(height)}', (p.get_x() + p.get_width() / 2., height),
                            ha='center', va='bottom', fontsize=9, xytext=(0, 2),
                            textcoords='offset points')

    # Hapus axes kosong & Atur Legenda
    if n_years < len(axes): fig.delaxes(axes[-1])
    handles = [plt.Rectangle((0,0),1,1, color=VISUAL_COLORS[label]) for label in VISUAL_COLORS]
    fig.legend(handles, VISUAL_COLORS.keys(), loc='upper center', bbox_to_anchor=(0.5, 0.95), ncol=2)

    # Simpan
    filename = f"Grafik_Distribusi_Outlier_{station_name.lower()}.png"
    plt.tight_layout(rect=[0, 0, 1, 0.94])
    plt.savefig(os.path.join(output_folder, filename), dpi=150, bbox_inches='tight')
    plt.close()
    print(f"   ✅ Grafik tersimpan: {filename}")

# ==============================================================================
# 4. EKSEKUSI UTAMA (MAIN LOOP)
# ==============================================================================

def main():
    print("="*80)
    print(" 💻 LANGKAH B: PRE-PROCESSING DATA DEBIT & KLASIFIKASI JENKS")
    print("="*80)

    try:
        mount_google_drive()
        print(f"\n📂 Folder Kerja: {GEE_EXPORT_FOLDER}")

        # Buat folder output visualisasi
        Report_Folder = os.path.join(GEE_EXPORT_FOLDER, 'Laporan_Analisis_Distribusi')
        Path(Report_Folder).mkdir(parents=True, exist_ok=True)

        for current_station in STATION_LIST:
            print(f"\n🔄 MEMPROSES STASIUN: {current_station.upper()}")
            print("-" * 40)

            roi_lower = current_station.lower()

            # --- A. Load Data Input ---
            file_citra = os.path.join(GEE_EXPORT_FOLDER, f'MASTER_TANGGAL_STRICT_{current_station.upper()}_2019_2024.csv')
            file_debit = os.path.join(GEE_EXPORT_FOLDER, f'debit_{roi_lower}_2019_2024.csv')

            # Fallback nama file jika format berbeda
            if not os.path.exists(file_citra):
                file_citra = file_citra.replace(current_station.upper(), roi_lower)

            if not os.path.exists(file_citra) or not os.path.exists(file_debit):
                print(f"❌ SKIP: File input untuk {current_station} tidak ditemukan.")
                continue

            try:
                df_citra = standardize_date_column(pd.read_csv(file_citra), 'date')
                df_debit = standardize_debit_column(pd.read_csv(file_debit), 'date', 'debit')
            except Exception as e:
                print(f"❌ Error saat membaca data: {e}")
                continue

            # --- B. Merge & Labeling Musim ---
            df_merged = pd.merge(df_citra, df_debit, on='Tanggal', how='inner')
            print(f"   -> Data Cocok (Citra & Debit): {len(df_merged)} baris")

            zom_code = ROI_TO_ZOM[current_station]
            intervals = [(pd.to_datetime(r['start']), pd.to_datetime(r['end']))
                         for r in RAINY_SEASON_DB if r['zom'] == zom_code]
            df_merged['Musim'] = df_merged['Tanggal'].apply(get_season_label, args=(intervals,))

            # --- C. Split Dataset (Latih vs Uji) ---
            df_latih = df_merged[(df_merged['Tanggal'] >= START_LATIH_STR) & (df_merged['Tanggal'] <= END_LATIH_STR)].copy()
            df_uji   = df_merged[(df_merged['Tanggal'] >= START_UJI_STR) & (df_merged['Tanggal'] <= END_UJI_STR)].copy()

            # --- D. Outlier Detection ---
            df_latih_tag = flag_outliers(df_latih, 'Debit_Mentah', IQR_MULT, Z_TH)
            df_uji_tag   = flag_outliers(df_uji, 'Debit_Mentah', IQR_MULT, Z_TH)

            # Visualisasi (Termasuk Outlier)
            visualize_distribution_with_outliers(pd.concat([df_latih_tag, df_uji_tag]), current_station, Report_Folder)

            # Filter Data Bersih
            df_latih_clean = df_latih_tag[~df_latih_tag['is_outlier']].rename(columns={'Debit_Mentah': 'Debit_Aktual'}).copy()
            df_uji_clean   = df_uji_tag[~df_uji_tag['is_outlier']].rename(columns={'Debit_Mentah': 'Debit_Aktual'}).copy()

            print(f"   -> Data Bersih: Latih={len(df_latih_clean)}, Uji={len(df_uji_clean)}")

            # --- E. Klasifikasi Jenks (Hanya Data Latih) ---
            threshold_candidates = {}

            # 1. Skenario Gabungan (Tanpa Musim)
            res_gabungan = calculate_jenks_breaks(df_latih_clean, 'Debit_Aktual', 'GABUNGAN')
            if res_gabungan:
                threshold_candidates['Jenks_Gabungan'] = {'TANPA_MUSIM': res_gabungan['Jenks']}

            # 2. Skenario Musiman
            df_h = df_latih_clean[df_latih_clean['Musim'] == 'Hujan']
            df_k = df_latih_clean[df_latih_clean['Musim'] == 'Kemarau']
            res_h = calculate_jenks_breaks(df_h, 'Debit_Aktual', 'HUJAN')
            res_k = calculate_jenks_breaks(df_k, 'Debit_Aktual', 'KEMARAU')

            if res_h or res_k:
                threshold_candidates['Jenks_Musiman'] = {
                    'Hujan': res_h['Jenks'] if res_h else {},
                    'Kemarau': res_k['Jenks'] if res_k else {}
                }

            # --- F. Simpan Ringkasan Global ---
            if res_gabungan:
                q1, q2 = res_gabungan['Jenks']['q1'], res_gabungan['Jenks']['q2']

                # Fungsi pembantu untuk loop
                def add_to_summary(df_segment, dataset_label):
                    if df_segment.empty: return
                    temp = df_segment.copy()
                    temp['Data_Set'] = dataset_label
                    temp['Stasiun'] = current_station
                    temp['Kelas_Debit'] = temp['Debit_Aktual'].apply(lambda x: assign_debit_class(x, q1, q2))

                    for _, row in temp.iterrows():
                        RINGKASAN_ROWS.append({
                            'Tanggal': row['Tanggal'],
                            'Data_Set': row['Data_Set'],
                            'Musim': row['Musim'],
                            'Nilai_Debit': row['Debit_Aktual'],
                            'Kelas_Debit': row['Kelas_Debit'],
                            'Stasiun': row['Stasiun']
                        })

                add_to_summary(df_latih_clean, 'Latih')
                add_to_summary(df_uji_clean, 'Uji')

            # --- G. Ekspor File Output ('_FIX') ---
            # 1. Threshold JSON
            with open(os.path.join(GEE_EXPORT_FOLDER, f'kandidat_thresholds_{roi_lower}_FIX.json'), 'w') as f:
                json.dump(threshold_candidates, f, indent=4)

            # 2. Data Debit Bersih
            df_latih_clean[['Tanggal', 'Debit_Aktual']].to_csv(os.path.join(GEE_EXPORT_FOLDER, f'data_debit_latih_{roi_lower}_{TAG_LATIH}_FIX.csv'), index=False)
            df_uji_clean[['Tanggal', 'Debit_Aktual']].to_csv(os.path.join(GEE_EXPORT_FOLDER, f'data_debit_uji_{roi_lower}_{TAG_UJI}_FIX.csv'), index=False)

            # 3. Daftar Tanggal untuk GEE (Langkah C)
            df_latih_clean[['Tanggal']].rename(columns={'Tanggal':'date'}).to_csv(os.path.join(GEE_EXPORT_FOLDER, f'tanggal_latih_{roi_lower}_{TAG_LATIH}_FIX.csv'), index=False)
            df_uji_clean[['Tanggal']].rename(columns={'Tanggal':'date'}).to_csv(os.path.join(GEE_EXPORT_FOLDER, f'tanggal_uji_{roi_lower}_{TAG_UJI}_FIX.csv'), index=False)

            print(f"   ✅ Ekspor data selesai untuk {current_station}.")

        # --- H. Ekspor Ringkasan Akhir (Tabel Bab 4) ---
        if RINGKASAN_ROWS:
            print("\n💾 Menyimpan File Ringkasan Global...")
            df_ringkasan = pd.DataFrame(RINGKASAN_ROWS).sort_values(by=['Stasiun', 'Data_Set', 'Tanggal'])

            # 1. Master Tabel Ringkasan
            df_ringkasan.to_csv(os.path.join(GEE_EXPORT_FOLDER, 'Ringkasan_Citra_Debit_Kelas_FIX.csv'), index=False)

            # 2. Tabel 4.1 (Jumlah Citra per Dataset)
            t41 = df_ringkasan.pivot_table(index='Stasiun', columns='Data_Set', values='Tanggal', aggfunc='count', fill_value=0)
            t41['Total'] = t41.sum(axis=1)
            t41.to_csv(os.path.join(GEE_EXPORT_FOLDER, 'Ringkasan_Tabel4_1_FIX.csv'))

            # 3. Tabel 4.2 (Detail Musim)
            t42 = df_ringkasan.pivot_table(index='Stasiun', columns=['Data_Set', 'Musim'], values='Tanggal', aggfunc='count', fill_value=0)
            t42.to_csv(os.path.join(GEE_EXPORT_FOLDER, 'Ringkasan_Tabel4_2_FIX.csv'))

            print("   ✅ Semua tabel ringkasan berhasil dibuat.")
        else:
            print("\n⚠️ Peringatan: Tidak ada data ringkasan yang dihasilkan.")

        print("\n" + "="*80)
        print("🎉 LANGKAH B SELESAI. SIAP LANJUT KE LANGKAH C (GEE EXTRACTION).")
        print("="*80)

    except Exception as e:
        print(f"\n❌ TERJADI ERROR FATAL: {e}")
        traceback.print_exc()

if __name__ == "__main__":
    main()